In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib as pl
import flopy

In [ ]:
# import containerized functionality from defaults.py
from defaults import *

In [ ]:
figure_path = pl.Path("figures")
figure_path.mkdir(parents=True, exist_ok=True)

### Parallel settings

1. Set voronoi to `True` to evaluate performance of Voronoi grid base and parallel models. metis cannot be `True` if voronoi is `True`.
2. Set metis to `True` to evaluate performance of structured grid base and parallel models split with Metis. voronoi cannot be `True` if metis is `True`.

In [ ]:
voronoi = False
metis = True

In [ ]:
paths = get_available_workspaces(metis=metis, voronoi=voronoi)
paths

In [ ]:
base_sim = flopy.mf6.MFSimulation.load(
    sim_name="basin",
    sim_ws=paths[0],
    verbosity_level=0,
)
base_gwf = base_sim.get_model()

In [ ]:
total_cells, active_cells = get_simulation_cell_count(base_sim)
total_cells, active_cells

In [ ]:
processors = get_simulation_processors(metis=metis, voronoi=voronoi)
processors

In [ ]:
mean_runtimes = []
mean_formulatetimes = []
mean_solutiontimes = []
mean_iterations = []
for path in paths:
    list_files = get_simulation_listfiles(path)
    mean_runtimes.append(
        np.mean(
            [
                SimulationData(list_file).get_model_runtime()
                for list_file in list_files
            ]
        )
    )
    mean_formulatetimes.append(
        np.mean(
            [
                SimulationData(list_file).get_formulate_time()
                for list_file in list_files
            ]
        )
    )
    mean_solutiontimes.append(
        np.mean(
            [
                SimulationData(list_file).get_solution_time()
                for list_file in list_files
            ]
        )
    )
    mean_iterations.append(
        np.mean(
            [
                SimulationData(list_file).get_total_iterations()
                for list_file in list_files
            ]
        )
    )

In [ ]:
total_memory_usage = []
max_memory_usage = []
for path in paths:
    list_files = get_simulation_listfiles(path)
    total_memory_usage.append(
        np.sum(
            [
                SimulationData(list_file).get_memory_usage()
                for list_file in list_files
            ]
        )
    )
    max_memory_usage.append(
        np.max(
            [
                SimulationData(list_file).get_memory_usage()
                for list_file in list_files
            ]
        )
    )

In [ ]:
speedup = mean_runtimes[0] / np.array(mean_runtimes)
formulate_speedup = mean_formulatetimes[0] / np.array(mean_formulatetimes)
solution_speedup = mean_solutiontimes[0] / np.array(mean_solutiontimes)

In [ ]:
if metis:
    split_string = "_metis"
else:
    split_string == ""
figure_base_name = f"{base_gwf.modelgrid.grid_type}{split_string}_{total_cells:,}_{active_cells:,}_"
figure_title = f"{base_gwf.modelgrid.grid_type.capitalize()} Grid -- {total_cells:,} Total Cells {active_cells:,} Active Cells"

In [ ]:
axd = plt.figure(
    layout="constrained",
    figsize=(figwidth, figheight * 0.9),
).subplot_mosaic(
    """
    ab
    cd
    """,
    empty_sentinel="X",
    sharex=True,
)

timing_data = {
    "a": mean_runtimes,
    "b": mean_iterations,
    "c": mean_formulatetimes,
    "d": mean_solutiontimes,
}
ylabels = {
    "a": "Runtime (sec)",
    "b": "Number of iterations",
    "c": "Formulation time (sec)",
    "d": "Solution time (sec)",
}

for idx, key in enumerate(axd.keys()):
    ax = axd[key]
    ax.plot(
        processors,
        timing_data[key],
        lw=0.5,
        color="black",
        marker="o",
        ms=4,
        mfc="none",
        mec="black",
    )
    ax.set_ylabel(ylabels[key])
    if key in ("c", "d"):
        ax.set_xlabel("Number of CPUs")

fig = plt.gcf()
fig.suptitle(figure_title)

fname = figure_path / f"{figure_base_name}timing.pdf"
fig.savefig(fname);

In [ ]:
cpu_range = (1, processors[-1])

axd = plt.figure(
    layout="constrained",
    figsize=(figwidth, figheight * 0.9),
).subplot_mosaic(
    """
    ab
    cd
    """,
    empty_sentinel="X",
    sharex=True,
)

speedup_data = {
    "a": formulate_speedup,
    "b": solution_speedup,
    "c": speedup,
    "d": max_memory_usage,
}
ylabels = {
    "a": "Formulate Speedup",
    "b": "Solution Speedup",
    "c": "Total Runtime Speedup",
    "d": "Maxmimum Memory Usage (GB)",
}

for idx, key in enumerate(axd.keys()):
    ax = axd[key]
    ax.plot(
        processors,
        speedup_data[key],
        lw=0.5,
        color="black",
        marker="o",
        ms=4,
        mfc="none",
        mec="black",
    )
    if key != "d":
        ax.plot(
            cpu_range, cpu_range, ls="--", lw=0.5, color="0.5", label="ideal"
        )
    ax.set_ylabel(ylabels[key])
    if key in ("c", "d"):
        ax.set_xlabel("Number of CPUs")
    ax.set_xlim(cpu_range)
    if key != "d":
        ax.set_ylim(cpu_range)
    if idx == 0:
        ax.legend(frameon=False)

fig = plt.gcf()
fig.suptitle(figure_title)

fname = figure_path / f"{figure_base_name}speedup.pdf"
fig.savefig(fname);

In [ ]:
axd = plt.figure(
    layout="constrained", figsize=(figwidth, figheight / 2)
).subplot_mosaic(
    """
    ab
    """,
    empty_sentinel="X",
)

speedup_data = {
    "a": total_memory_usage,
    "b": max_memory_usage,
}
ylabels = {"a": "Total Memory Usage (GB)", "b": "Maxmimum Memory Usage (GB)"}

for idx, key in enumerate(axd.keys()):
    ax = axd[key]
    ax.plot(
        processors,
        speedup_data[key],
        lw=0.5,
        color="black",
        marker="o",
        ms=4,
        mfc="none",
        mec="black",
    )
    ax.set_ylabel(ylabels[key])
    ax.set_xlabel("Number of CPUs")
    ax.set_xlim(cpu_range)

fig = plt.gcf()
fig.suptitle(figure_title)

fname = figure_path / f"{figure_base_name}memory.pdf"
fig.savefig(fname);